# Wrangle Data with WeRateDogs through Twitter API


**Goal:** wrangle WeRateDogs Twitter data to create interesting and trustworthy analyses and visualizations.

The Twitter archive is great, but it only contains very basic tweet information. Additional gathering, then assessing and cleaning is required for "Wow!"-worthy analyses and visualizations.

Find at least **8 quality issues** and **2 tidiness issues**. Provide at least **3 insights** and **1 visualization** after data wrangling.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import os
import json
from datetime import datetime

%matplotlib inline

In [588]:
import dill
dill.dump_session('notebook_session.db')
#dill.load_session('notebook_session.db')

# Gather

In [2]:
#read in the WeRateDogs twitter archive
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
#install breed predictions of a neural network for dogs
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
r = requests.get(url)

with open(os.path.join('./', url.split('/')[-1]), mode='wb') as file:
    file.write(r.content)

In [4]:
image_predictions = pd.read_csv('image-predictions.tsv', sep='\t')

So for the last row under `image-predictions.tsv`:

- tweet_id is the last part of the tweet URL after "status/" → https://twitter.com/dog_rates/status/889531135344209921

- p1 is the algorithm's #1 prediction for the image in the tweet → **golden retriever**

- p1_conf is how confident the algorithm is in its #1 prediction → **95%**

- p1_dog is whether or not the #1 prediction is a breed of dog → **TRUE**

- p2 is the algorithm's second most likely prediction → **Labrador retriever**

- p2_conf is how confident the algorithm is in its #2 prediction → **1%**

- p2_dog is whether or not the #2 prediction is a breed of dog → **TRUE**
etc.

> Use Tweepy (Twitter API) for each tweet's JSON data using the tweet IDs under the WeRateDogs archive.

In [5]:
#handle environment variables
from os.path import join, dirname
from dotenv import load_dotenv

path = './weratedogs-env-vars.env'
load_dotenv(dotenv_path = path, verbose = True)

True

In [6]:
#create API object
import tweepy

consumer_key = os.getenv('CONSUMER_KEY')
consumer_secret = os.getenv('CONSUMER_SECRET')
access_token = os.getenv('TOKEN')
access_secret = os.getenv('TOKEN_SECRET')

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [45]:
#save JSON tweet data to tweet_json.txt

tweet_ids = twitter_archive.tweet_id
num_success = 0
num_errors = 0

start_time = datetime.now()

with open('error_file.txt', 'w', encoding='utf-8') as errfile:
    with open('tweet_json.txt', 'w', encoding='utf-8') as outfile:
        for tweet_id in tweet_ids:
            try: 
                tweet = api.get_status(tweet_id)
                tweet_json = json.dumps(tweet._json)
                json.dump(tweet_json, outfile)
                outfile.write('\n')
                num_success += 1
                print(tweet_id)
            except tweepy.TweepError as e:
                err_string = '{}: {}'.format(tweet_id, e.args[0][0]['message'])
                errfile.write(err_string)
                errfile.write('\n')
                num_errors += 1
                print(err_string)
                continue
        
time_elapsed = datetime.now() - start_time

print('Time elapsed (hh:mm:ss:ms): {}'.format(time_elapsed))
print('Valid tweets: {}'.format(num_success))
print('Invalid tweets: {}'.format(num_errors))

892420643555336193
892177421306343426
891815181378084864
891689557279858688
891327558926688256
891087950875897856
890971913173991426
890729181411237888
890609185150312448
890240255349198849
890006608113172480
889880896479866881
889665388333682689
889638837579907072
889531135344209921
889278841981685760
888917238123831296
888804989199671297
888554962724278272
888202515573088257: No status found with that ID.
888078434458587136
887705289381826560
887517139158093824
887473957103951883
887343217045368832
887101392804085760
886983233522544640
886736880519319552
886680336477933568
886366144734445568
886267009285017600
886258384151887873
886054160059072513
885984800019947520
885528943205470208
885518971528720385
885311592912609280
885167619883638784
884925521741709313
884876753390489601
884562892145688576
884441805382717440
884247878851493888
884162670584377345
883838122936631299
883482846933004288
883360690899218434
883117836046086144
882992080364220416
882762694511734784
882627270321602560


823719002937630720
823699002998870016
823581115634085888
823333489516937216
823322678127919110
823269594223824897
822975315408461824
822872901745569793
822859134160621569
822647212903690241
822610361945911296
822489057087389700
822462944365645825
822244816520155136
822163064745328640
821886076407029760
821813639212650496
821765923262631936
821522889702862852
821421320206483457
821407182352777218
821153421864615936
821149554670182400
821107785811234820
821044531881721856
820837357901512704
820749716845686786
820690176645140481
820494788566847489
820446719150292993
820314633777061888
820078625395449857
820013781606658049
819952236453363712
819924195358416896
819711362133872643
819588359383371776
819347104292290561
819238181065359361
819227688460238848
819015337530290176
819015331746349057
819006400881917954
819004803107983360
818646164899774465
818627210458333184
818614493328580609
818588835076603904
818536468981415936
818307523543449600
818259473185828864
818145370475810820
817908911860

Rate limit reached. Sleeping for: 362


787810552592695296
787717603741622272
787397959788929025
787322443945877504
787111942498508800
786963064373534720
786729988674449408
786709082849828864
786664955043049472
786595970293370880
786363235746385920
786286427768250368
786233965241827333
786051337297522688
786036967502913536
785927819176054784
785872687017132033
785639753186217984
785533386513321988
785515384317313025
785264754247995392
785170936622350336
784826020293709826
784517518371221505
784431430411685888
784183165795655680
784057939640352768
783839966405230592
783821107061198850
783695101801398276
783466772167098368
783391753726550016
783347506784731136
783334639985389568
783085703974514689
782969140009107456
782747134529531904
782722598790725632
782598640137187329
782305867769217024
782021823840026624
781955203444699136
781661882474196992
781655249211752448
781524693396357120
781308096455073793
781251288990355457
781163403222056960
780931614150983680
780858289093574656
780800785462489090
780601303617732608
780543529827

732726085725589504
732585889486888962
732375214819057664
732005617171337216
731285275100512256
731156023742988288
730924654643314689
730573383004487680
730427201120833536
730211855403241472
730196704625098752
729854734790754305
729838605770891264
729823566028484608
729463711119904772
729113531270991872
728986383096946689
728760639972315136
728751179681943552
728653952833728512
728409960103686147
728387165835677696
728046963732717569
728035342121635841
728015554473250816
727685679342333952
727644517743104000
727524757080539137
727314416056803329
727286334147182592
727175381690781696
727155742655025152
726935089318363137
726887082820554753
726828223124897792
726224900189511680
725842289046749185
725786712245440512
725729321944506368
725458796924002305
724983749226668032
724771698126512129
724405726123311104
724049859469295616
724046343203856385
724004602748780546
723912936180330496
723688335806480385
723673163800948736
723179728551723008
722974582966214656
722613351520608256
721503162398

689280876073582592
689275259254616065
689255633275777024
689154315265683456
689143371370250240
688916208532455424
688908934925697024
688898160958271489
688894073864884227
688828561667567616
688804835492233216
688789766343622656
688547210804498433
688519176466644993
688385280030670848
688211956440801280
688179443353796608
688116655151435777
688064179421470721
687841446767013888
687826841265172480
687818504314159109
687807801670897665
687732144991551489
687704180304273409
687664829264453632
687494652870668288
687480748861947905
687476254459715584
687460506001633280
687399393394311168
687317306314240000
687312378585812992
687127927494963200
687124485711986689
687109925361856513
687102708889812993
687096057537363968
686947101016735744
686760001961103360
686749460672679938


Rate limit reached. Sleeping for: 428


686730991906516992
686683045143953408
686618349602762752
686606069955735556
686394059078897668
686386521809772549
686377065986265092
686358356425093120
686286779679375361
686050296934563840
686035780142297088
686034024800862208
686007916130873345
686003207160610816
685973236358713344
685943807276412928
685906723014619143
685681090388975616
685667379192414208
685663452032069632
685641971164143616
685547936038666240
685532292383666176
685325112850124800
685321586178670592
685315239903100929
685307451701334016
685268753634967552
685198997565345792
685169283572338688
684969860808454144
684959798585110529
684940049151070208
684926975086034944
684914660081053696
684902183876321280
684880619965411328
684830982659280897
684800227459624960
684594889858887680
684588130326986752
684567543613382656
684538444857667585
684481074559381504
684460069371654144
684241637099323392
684225744407494656
684222868335505415
684200372118904832
684195085588783105
684188786104872960
684177701129875456
684147889187

672139350159835138
672125275208069120
672095186491711488
672082170312290304
672068090318987265
671896809300709376
671891728106971137
671882082306625538
671879137494245376
671874878652489728
671866342182637568
671855973984772097
671789708968640512
671768281401958400
671763349865160704
671744970634719232
671743150407421952
671735591348891648
671729906628341761
671561002136281088
671550332464455680
671547767500775424
671544874165002241
671542985629241344
671538301157904385
671536543010570240
671533943490011136
671528761649688577
671520732782923777
671518598289059840
671511350426865664
671504605491109889
671497587707535361
671488513339211776
671486386088865792
671485057807351808
671390180817915904
671362598324076544
671357843010908160
671355857343524864
671347597085433856
671186162933985280
671182547775299584
671166507850801152
671163268581498880
671159727754231808
671154572044468225
671151324042559489
671147085991960577
671141549288370177
671138694582165504
671134062904504320
671122204919

In [7]:
#read tweet_json data into a list of tweet JSONs
all_tweets = []

for line in open('tweet_json.txt', 'r', encoding = 'utf-8'):
    json_data = json.loads(json.loads(line))
    all_tweets.append(json_data)

In [8]:
#pick interesting fields to add to DataFrame
all_tweets[0]

{'created_at': 'Tue Aug 01 16:23:56 +0000 2017',
 'id': 892420643555336193,
 'id_str': '892420643555336193',
 'text': "This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU",
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 892420639486877696,
    'id_str': '892420639486877696',
    'indices': [86, 109],
    'media_url': 'http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
    'url': 'https://t.co/MgUWQ76dJU',
    'display_url': 'pic.twitter.com/MgUWQ76dJU',
    'expanded_url': 'https://twitter.com/dog_rates/status/892420643555336193/photo/1',
    'type': 'photo',
    'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'medium': {'w': 540, 'h': 528, 'resize': 'fit'},
     'small': {'w': 540, 'h': 528, 'resize': 'fit'},
     'large': {'w': 540, 'h': 528, 'resize': 'fit'}}}]},


In [9]:
#create dictionary to use in pandas DataFrame
tweets_json = []
for tweet in all_tweets:
    tweets_json.append({'tweet_id': tweet['id'],
                     'created_at': tweet['created_at'],
                      'favorite_count': tweet['favorite_count'],
                      'retweet_count': tweet['retweet_count']
                     })

In [10]:
tweets_json[0]

{'tweet_id': 892420643555336193,
 'created_at': 'Tue Aug 01 16:23:56 +0000 2017',
 'favorite_count': 37424,
 'retweet_count': 8130}

In [11]:
#create Pandas DataFrame
tweet_fields = pd.DataFrame(tweets_json, columns = ['tweet_id', 'created_at', 'favorite_count', 'retweet_count'])

In addition to `tweet_id`, `created_at`, and `retweet_count`, I also decided to add the additional fields `created_at` and `followers_count` for the user. This is because I want to examine how the other variables such as `favorite_count` and `retweet_count` could impact the fluctuation of followers, and also see which time of the day under `created_at` gives the biggest spike in followers.

# Assess

In [12]:
twitter_archive.sample(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2172,669327207240699904,NaN,NaN,2015-11-25 01:30:16 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Just got home from college. Dis my dog. She do...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/669327207...,13,10,None,None,None,None,None
706,785533386513321988,NaN,NaN,2016-10-10 17:32:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Dallas. Her tongue is ridiculous. 11/1...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/785533386...,11,10,Dallas,None,None,None,None
853,765371061932261376,NaN,NaN,2016-08-16 02:14:15 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Louie. He's had a long day. Did a lot ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/765371061...,11,10,Louie,None,None,pupper,None
130,867072653475098625,NaN,NaN,2017-05-23 17:40:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @rachaeleasler: these @dog_rates hats are 1...,8.650134e+17,7.874618e+17,2017-05-18 01:17:25 +0000,https://twitter.com/rachaeleasler/status/86501...,13,10,None,None,None,None,None
1527,690360449368465409,NaN,NaN,2016-01-22 02:28:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Stop sending in lobsters. This is the final wa...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/690360449...,9,10,the,None,None,None,None
961,751132876104687617,NaN,NaN,2016-07-07 19:16:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cooper. He's just so damn happy. 10/10...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/751132876...,10,10,Cooper,None,None,None,puppo
280,839549326359670784,NaN,NaN,2017-03-08 18:52:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Winston. He knows he's a little too big f...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/839549326...,12,10,Winston,None,None,None,None
1047,743510151680958465,NaN,NaN,2016-06-16 18:26:48 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","This is Beya. She doesn't want to swim, so she...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/743510151...,13,10,Beya,None,None,None,None
312,835264098648616962,NaN,NaN,2017-02-24 23:04:14 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Lola. Her hobbies include being precious ...,NaN,NaN,NaN,https://www.gofundme.com/lolas-life-saving-sur...,12,10,Lola,None,None,None,None
1679,682032003584274432,NaN,NaN,2015-12-30 02:54:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Ulysses. He likes holding hands and hi...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/682032003...,11,10,Ulysses,None,None,None,None


In [13]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [14]:
#look at denominators that are not equal to 10
twitter_archive.query('rating_denominator != 10').head(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
313,835246439529840640,8.352460e+17,26259576.0,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,NaN,NaN,NaN,NaN,960,0,None,None,None,None,None
342,832088576586297345,8.320875e+17,30582082.0,2017-02-16 04:45:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@docmisterio account started on 11/15/15,NaN,NaN,NaN,NaN,11,15,None,None,None,None,None
433,820690176645140481,NaN,NaN,2017-01-15 17:52:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",The floofs have been released I repeat the flo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/820690176...,84,70,None,None,None,None,None
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,NaN,NaN,NaN,"https://www.gofundme.com/sams-smile,https://tw...",24,7,Sam,None,None,None,None
784,775096608509886464,NaN,NaN,2016-09-11 22:20:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: After so many requests, this is...",7.403732e+17,4.196984e+09,2016-06-08 02:41:38 +0000,https://twitter.com/dog_rates/status/740373189...,9,11,None,None,None,None,None


In [18]:
#check counts of denominators != 10
twitter_archive.rating_denominator.value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [28]:
#check corresponding text with denominators != 10
denoms = twitter_archive[twitter_archive['rating_denominator'] != 10][['tweet_id', 'text', 'rating_numerator', 'rating_denominator']]
pd.set_option('display.max_colwidth', -1)
denoms

,tweet_id,text,rating_numerator,rating_denominator
313,835246439529840640,"@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho",960,0
342,832088576586297345,@docmisterio account started on 11/15/15,11,15
433,820690176645140481,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,84,70
516,810984652412424192,Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx,24,7
784,775096608509886464,"RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…",9,11
902,758467244762497024,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,165,150
1068,740373189193256964,"After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ",9,11
1120,731156023742988288,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv,204,170
1165,722974582966214656,Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a,4,20
1202,716439118184652801,This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq,50,50


In [73]:
#check non-int numerators
twitter_archive.query('rating_numerator == 75 or rating_numerator == 26')[['tweet_id', 'text', 'rating_numerator']]

,tweet_id,text,rating_numerator
340,832215909146226688,"RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wu…",75
695,786709082849828864,"This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",75
1712,680494726643068929,Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,26


In [30]:
#look at nonsensical dog names
mask = twitter_archive.name.str.islower()
bad_names = twitter_archive.loc[mask]['name'].sample(50)
bad_names.sample(50)

118     quite     
2198    a         
2346    the       
2347    a         
2314    a         
2116    a         
1206    old       
2326    quite     
2287    a         
1955    a         
369     one       
2354    a         
2034    a         
2066    a         
1877    a         
1382    a         
2218    a         
1002    a         
1193    a         
56      a         
1361    a         
1071    getting   
2255    a         
1878    a         
988     not       
822     just      
1017    a         
1040    actually  
2222    a         
1853    a         
2264    a         
2146    a         
1499    a         
22      such      
2345    the       
2273    a         
542     incredibly
1916    life      
1923    a         
1457    just      
335     not       
1724    by        
682     mad       
2335    an        
1785    a         
2204    an        
1120    this      
1049    a         
759     an        
1435    getting   
Name: name, dtype: object

In [562]:
#check for number of dogs that fall under more than 1 dog stage
def mask_duplicate_stages(df):
    stage_mask = []
    for i, row in df.iterrows():
        sum = 0
        if (row['doggo'] != 'None'):
            sum += 1
        if (row['floofer'] != 'None'):
            sum += 1
        if (row['pupper'] != 'None'):
            sum += 1
        if (row['puppo'] != 'None'):
            sum += 1

        if sum > 1:
            stage_mask.append(True)
        else:
            stage_mask.append(False)
    
    return stage_mask

In [560]:
def mask_single_stages(df):
    stage_mask = []
    for i, row in df.iterrows():
        sum = 0
        if (row['doggo'] != 'None'):
            sum += 1
        if (row['floofer'] != 'None'):
            sum += 1
        if (row['pupper'] != 'None'):
            sum += 1
        if (row['puppo'] != 'None'):
            sum += 1

        if sum == 1:
            stage_mask.append(True)
        else:
            stage_mask.append(False)
    
    return stage_mask

In [561]:
def mask_all_stages(df):
    stage_mask = []
    for i, row in df.iterrows():
        sum = 0
        if (row['doggo'] != 'None'):
            sum += 1
        if (row['floofer'] != 'None'):
            sum += 1
        if (row['pupper'] != 'None'):
            sum += 1
        if (row['puppo'] != 'None'):
            sum += 1

        if sum > 0:
            stage_mask.append(True)
        else:
            stage_mask.append(False)
    
    return stage_mask

In [566]:
#create masks
dup_stage_mask = mask_duplicate_stages(twitter_archive)    
single_stage_mask = mask_single_stages(twitter_archive)
all_stage_mask = mask_all_stages(twitter_archive)

In [567]:
#number of dogs with more than 1 stage
twitter_archive[dup_stage_mask].shape[0]

14

In [568]:
#number of dogs with only 1 stage
twitter_archive[single_stage_mask].shape[0]

366

In [569]:
#number of dogs with >= 1 stage
twitter_archive[all_stage_mask].shape[0]

380

In [487]:
#check for text associated with each dog showing repeated stage
twitter_archive[dup_stage_mask][['tweet_id', 'text', 'doggo', 'floofer', 'pupper', 'puppo']]

,tweet_id,text,doggo,floofer,pupper,puppo
191,855851453814013952,Here's a puppo participating in the #ScienceMarch. Cleverly disguising her own doggo agenda. 13/10 would keep the planet habitable for https://t.co/cMhq16isel,doggo,None,None,puppo
200,854010172552949760,"At first I thought this was a shy doggo, but it's actually a Rare Canadian Floofer Owl. Amateurs would confuse the two. 11/10 only send dogs https://t.co/TXdT3tmuYk",doggo,floofer,None,None
460,817777686764523521,"This is Dido. She's playing the lead role in ""Pupper Stops to Catch Snow Before Resuming Shadow Box with Dried Apple."" 13/10 (IG: didodoggo) https://t.co/m7isZrOBX7",doggo,None,pupper,None
531,808106460588765185,Here we have Burke (pupper) and Dexter (doggo). Pupper wants to be exactly like doggo. Both 12/10 would pet at same time https://t.co/ANBpEYHaho,doggo,None,pupper,None
565,802265048156610565,"Like doggo, like pupper version 2. Both 11/10 https://t.co/9IxWAXFqze",doggo,None,pupper,None
575,801115127852503040,This is Bones. He's being haunted by another doggo of roughly the same size. 12/10 deep breaths pupper everything's fine https://t.co/55Dqe0SJNj,doggo,None,pupper,None
705,785639753186217984,This is Pinot. He's a sophisticated doggo. You can tell by the hat. Also pointier than your average pupper. Still 10/10 would pet cautiously https://t.co/f2wmLZTPHd,doggo,None,pupper,None
733,781308096455073793,"Pupper butt 1, Doggo 0. Both 12/10 https://t.co/WQvcPEpH2u",doggo,None,pupper,None
778,775898661951791106,"RT @dog_rates: Like father (doggo), like son (pupper). Both 12/10 https://t.co/pG2inLaOda",doggo,None,pupper,None
822,770093767776997377,RT @dog_rates: This is just downright precious af. 12/10 for both pupper and doggo https://t.co/o5J479bZUC,doggo,None,pupper,None


In [134]:
#check for retweets and duplicates
twitter_archive[~(twitter_archive.retweeted_status_id.isnull())]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
19,888202515573088257,NaN,NaN,2017-07-21 01:02:36 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: This is Canela. She attempted some fancy porch pics. They were unsuccessful. 13/10 someone help her https://t.co/cLyzpcUcMX,8.874740e+17,4.196984e+09,2017-07-19 00:47:34 +0000,"https://twitter.com/dog_rates/status/887473957103951883/photo/1,https://twitter.com/dog_rates/status/887473957103951883/photo/1,https://twitter.com/dog_rates/status/887473957103951883/photo/1,https://twitter.com/dog_rates/status/887473957103951883/photo/1",13,10,Canela,None,None,None,None
32,886054160059072513,NaN,NaN,2017-07-15 02:45:48 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @Athletics: 12/10 #BATP https://t.co/WxwJmvjfxo,8.860537e+17,1.960740e+07,2017-07-15 02:44:07 +0000,"https://twitter.com/dog_rates/status/886053434075471873,https://twitter.com/dog_rates/status/886053434075471873",12,10,None,None,None,None,None
36,885311592912609280,NaN,NaN,2017-07-13 01:35:06 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: This is Lilly. She just parallel barked. Kindly requests a reward now. 13/10 would pet so well https://t.co/SATN4If5H5,8.305833e+17,4.196984e+09,2017-02-12 01:04:29 +0000,"https://twitter.com/dog_rates/status/830583320585068544/photo/1,https://twitter.com/dog_rates/status/830583320585068544/photo/1,https://twitter.com/dog_rates/status/830583320585068544/photo/1,https://twitter.com/dog_rates/status/830583320585068544/photo/1",13,10,Lilly,None,None,None,None
68,879130579576475649,NaN,NaN,2017-06-26 00:13:58 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: This is Emmy. She was adopted today. Massive round of pupplause for Emmy and her new family. 14/10 for all involved https://…,8.780576e+17,4.196984e+09,2017-06-23 01:10:23 +0000,"https://twitter.com/dog_rates/status/878057613040115712/photo/1,https://twitter.com/dog_rates/status/878057613040115712/photo/1",14,10,Emmy,None,None,None,None
73,878404777348136964,NaN,NaN,2017-06-24 00:09:53 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","RT @dog_rates: Meet Shadow. In an attempt to reach maximum zooming borkdrive, he tore his ACL. Still 13/10 tho. Help him out below\n\nhttps:/…",8.782815e+17,4.196984e+09,2017-06-23 16:00:04 +0000,"https://www.gofundme.com/3yd6y1c,https://twitter.com/dog_rates/status/878281511006478336/photo/1",13,10,Shadow,None,None,None,None
74,878316110768087041,NaN,NaN,2017-06-23 18:17:33 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: Meet Terrance. He's being yelled at because he stapled the wrong stuff together. 11/10 hang in there Terrance https://t.co/i…,6.690004e+17,4.196984e+09,2015-11-24 03:51:38 +0000,https://twitter.com/dog_rates/status/669000397445533696/photo/1,11,10,Terrance,None,None,None,None
78,877611172832227328,NaN,NaN,2017-06-21 19:36:23 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @rachel2195: @dog_rates the boyfriend and his soaking wet pupper h*cking love his new hat 14/10 https://t.co/dJx4Gzc50G,8.768508e+17,5.128045e+08,2017-06-19 17:14:49 +0000,"https://twitter.com/rachel2195/status/876850772322988033/photo/1,https://twitter.com/rachel2195/status/876850772322988033/photo/1,https://twitter.com/rachel2195/status/876850772322988033/photo/1,https://twitter.com/rachel2195/status/876850772322988033/photo/1",14,10,None,None,None,pupper,None
91,874434818259525634,NaN,NaN,2017-06-13 01:14:41 +0000,"<a href=""http://twitter.com/download/iphone"" re

In [32]:
#check 1 instance to verify that these retweets are, in fact, duplicates
twitter_archive[twitter_archive.text.str.contains('Canela')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
19,888202515573088257,NaN,NaN,2017-07-21 01:02:36 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: This is Canela. She attempted some fancy porch pics. They were unsuccessful. 13/10 someone help her https://t.co/cLyzpcUcMX,8.874740e+17,4.196984e+09,2017-07-19 00:47:34 +0000,"https://twitter.com/dog_rates/status/887473957103951883/photo/1,https://twitter.com/dog_rates/status/887473957103951883/photo/1,https://twitter.com/dog_rates/status/887473957103951883/photo/1,https://twitter.com/dog_rates/status/887473957103951883/photo/1",13,10,Canela,None,None,None,None
23,887473957103951883,NaN,NaN,2017-07-19 00:47:34 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Canela. She attempted some fancy porch pics. They were unsuccessful. 13/10 someone help her https://t.co/cLyzpcUcMX,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/887473957103951883/photo/1,https://twitter.com/dog_rates/status/887473957103951883/photo/1",13,10,Canela,None,None,None,None


In [136]:
image_predictions.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1404,769695466921623552,https://pbs.twimg.com/media/Cq6B8V6XYAA1T1R.jpg,1,pug,0.407117,True,muzzle,0.165638,False,kuvasz,0.045837,True
1304,753294487569522689,https://pbs.twimg.com/media/CnQ9Vq1WEAEYP01.jpg,1,chow,0.194773,True,monitor,0.102305,False,Siberian_husky,0.086855,True
557,677687604918272002,https://pbs.twimg.com/media/CWehRdEWIAAySyO.jpg,1,Pembroke,0.573047,True,sunglasses,0.126758,False,golden_retriever,0.108047,True
413,673956914389192708,https://pbs.twimg.com/media/CVpgPGwWoAEV7gG.jpg,1,pug,0.586161,True,Brabancon_griffon,0.082744,True,Chihuahua,0.045878,True
36,666447344410484738,https://pbs.twimg.com/media/CT-yU5QWwAEjLX5.jpg,1,curly-coated_retriever,0.322084,True,giant_schnauzer,0.287955,True,Labrador_retriever,0.166331,True
1105,722974582966214656,https://pbs.twimg.com/media/CgiFjIpWgAA4wVp.jpg,1,Great_Dane,0.246762,True,Greater_Swiss_Mountain_dog,0.126131,True,Weimaraner,0.085297,True
1048,713411074226274305,https://pbs.twimg.com/media/CeaLlAPUMAIcC7U.jpg,1,Great_Pyrenees,0.720337,True,Samoyed,0.129542,True,kuvasz,0.122451,True
521,676575501977128964,https://pbs.twimg.com/media/CWOt07EUsAAnOYW.jpg,1,feather_boa,0.424106,False,Yorkshire_terrier,0.073144,True,Shetland_sheepdog,0.057598,True
802,691459709405118465,https://pbs.twimg.com/media/CZiO7mWUEAAa4zo.jpg,1,Shetland_sheepdog,0.551206,True,collie,0.232544,True,Border_collie,0.095218,True
485,675489971617296384,https://pbs.twimg.com/media/CV_SimUWoAAvJSY.jpg,1,West_Highland_white_terrier,0.139613,True,seat_belt,0.118647,False,Old_English_sheepdog,0.093952,True


In [137]:
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [138]:
image_predictions.shape[0]

2075

In [355]:
#check for tweet_ids in common between image_predictions and twitter_archive
def find_common_tweet_ids(df1, df2):
    common = df1.merge(df2, on = ['tweet_id'])
    return common

In [356]:
#check for tweet_ids NOT in common (missing from image_predictions)
def find_missing_tweet_ids(df1, df2):
    missing = df1[(~df1.tweet_id.isin(df2.tweet_id))]
    return missing

In [368]:
#check for missing tweet_ids between twitter_archive and image_predictions
common = find_common_tweet_ids(twitter_archive, image_predictions)
missing = find_missing_tweet_ids(twitter_archive, image_predictions)

In [369]:
print(common.tweet_id.tail(5))
print(missing.tweet_id.tail(5))

2070    666049248165822465
2071    666044226329800704
2072    666033412701032449
2073    666029285002620928
2074    666020888022790149
Name: tweet_id, dtype: int64
2038    671550332464455680
2149    669684865554620416
2189    668967877119254528
2212    668587383441514497
2298    667070482143944705
Name: tweet_id, dtype: int64


In [371]:
#check if the sum of len(common) and len(missing) equals the entire length of twitter_archive
len(twitter_archive), len(common) + len(missing)

(2356, 2356)

In [372]:
#check for missing tweet_ids between twitter_archive and tweet_fields
common1 = find_common_tweet_ids(twitter_archive, tweet_fields)
missing1 = find_missing_tweet_ids(twitter_archive, tweet_fields)

In [373]:
print(common1.tweet_id.tail(5))
print(missing1.tweet_id.tail(5))

2329    666049248165822465
2330    666044226329800704
2331    666033412701032449
2332    666029285002620928
2333    666020888022790149
Name: tweet_id, dtype: int64
566     802247111496568832
784     775096608509886464
818     770743923962707968
932     754011816964026368
1726    680055455951884288
Name: tweet_id, dtype: int64


In [374]:
#check if the sum of len(common1) and len(missing1) equals the entire length of twitter_archive
len(twitter_archive), len(common1) + len(missing1)

(2356, 2356)

In [375]:
#check for missing ids in common between image_predictions and tweet_fields that are not present in twitter_archive
common3 = find_common_tweet_ids(missing, missing1)

In [376]:
print(common3.tweet_id)

0    872668790621863937
1    869988702071779329
2    866816280283807744
3    856602993587888130
4    845459076796616705
5    827228250799742977
6    812747805718642688
7    775096608509886464
8    770743923962707968
Name: tweet_id, dtype: int64


In [378]:
#check number of missing ids in common
len(common3)

9

In [140]:
image_predictions.query('p1_conf >= 0.95')

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,1.459380e-02,False,golden_retriever,7.958960e-03,True
17,666104133288665088,https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg,1,hen,0.965932,False,cock,3.391940e-02,False,partridge,5.206580e-05,False
25,666362758909284353,https://pbs.twimg.com/media/CT9lXGsUcAAyUFt.jpg,1,guinea_pig,0.996496,False,skunk,2.402450e-03,False,hamster,4.608630e-04,False
27,666396247373291520,https://pbs.twimg.com/media/CT-D2ZHWIAA3gK1.jpg,1,Chihuahua,0.978108,True,toy_terrier,9.396970e-03,True,papillon,4.576810e-03,True
40,666691418707132416,https://pbs.twimg.com/media/CUCQTpEWEAA7EDz.jpg,1,German_shepherd,0.975401,True,beagle,8.687270e-03,True,bloodhound,5.394040e-03,True
45,666786068205871104,https://pbs.twimg.com/media/CUDmZIkWcAAIPPe.jpg,1,snail,0.999888,False,slug,5.514170e-05,False,acorn,2.625800e-05,False
53,667012601033924608,https://pbs.twimg.com/media/CUG0bC0U8AAw2su.jpg,1,hyena,0.987230,False,African_hunting_dog,1.260080e-02,False,coyote,5.735010e-05,False
58,667090893657276420,https://pbs.twimg.com/media/CUH7oLuUsAELWib.jpg,1,Chihuahua,0.959514,True,Italian_greyhound,5.370150e-03,True,Pomeranian,2.641330e-03,True
76,667435689202614272,https://pbs.twimg.com/media/CUM1OHCW4AEgGSi.jpg,1,Rottweiler,0.999091,True,miniature_pinscher,4.503550e-04,True,black-and-tan_coonhound,1.571400e-04,True
77,667437278097252352,https://pbs.twimg.com/media/CUM2qWaWoAUZ06L.jpg,1,porcupine,0.989154,False,bath_towel,6.300490e-03,False,badger,9.663400e-04,False


In [75]:
#check if timestamps in twitter archive and scraped tweets are consistent
time_const = twitter_archive[['tweet_id', 'timestamp']]
time_const = pd.merge(time_const, tweet_fields, on = 'tweet_id', how = 'left')

time_const.sample(20)

,tweet_id,timestamp,created_at,favorite_count,retweet_count
1827,676430933382295552,2015-12-14 15:57:56 +0000,Mon Dec 14 15:57:56 +0000 2015,1415.0,357.0
761,778286810187399168,2016-09-20 17:36:50 +0000,Tue Sep 20 17:36:50 +0000 2016,10892.0,3515.0
999,748220828303695873,2016-06-29 18:25:21 +0000,Wed Jun 29 18:25:21 +0000 2016,14631.0,8200.0
178,857263160327368704,2017-04-26 16:00:39 +0000,Wed Apr 26 16:00:39 +0000 2017,19939.0,4533.0
2103,670676092097810432,2015-11-28 18:50:15 +0000,Sat Nov 28 18:50:15 +0000 2015,245.0,43.0
1718,680176173301628928,2015-12-25 00:00:11 +0000,Fri Dec 25 00:00:11 +0000 2015,4010.0,1641.0
1003,747844099428986880,2016-06-28 17:28:22 +0000,Tue Jun 28 17:28:22 +0000 2016,2904.0,785.0
2085,670804601705242624,2015-11-29 03:20:54 +0000,Sun Nov 29 03:20:54 +0000 2015,1951.0,967.0
1512,691321916024623104,2016-01-24 18:09:23 +0000,Sun Jan 24 18:09:23 +0000 2016,2650.0,684.0
936,753375668877008896,2016-07-13 23:48:51 +0000,Wed Jul 13 23:48:51 +0000 2016,7923.0,2456.0


## Quality ##

**`twitter_archive`**:

**Missing Data**

- `twitter_archive` contains 281 tweets that are missing from `image_predictions`.
- `twitter_archive` contains 22 tweets that are missing from `tweet_fields`. After dropping previous entries, there should be 13 tweets to be dropped.


- retweeted data indicates duplicated tweets.
- unnecessary anchor tags under **source**
- erroneous datatypes under **in_reply_to_status_id**, **in_reply_to_user_id**, **retweeted_status_id**, **retweeted_status_user_id**, **timestamp**, and **retweeted_status_timestamp**.
- inconsistent & inaccurate readings mapped from text to numerator & denominator fields (e.g. 24/7 being mistaken as a rating).
- some numerator values contain a decimal point (e.g. 9.75/10).
- **rating_denominator** has values besides 10
- nonsensical dog names such as *a, an, quite, by, actually, such, not, one*.
- inconsistent timestamp format between `twitter_archive` and `tweet_fields`.
- there is one entry with no rating.

## Tidiness

**`twitter_archive`**:

- **doggo**, **floofer**, **pupper**, and **puppo** all are not under 1 category variable.
- **favorite_count** and **retweet_count** fields under `tweet_fields` need to be merged with `twitter_archive`.
- **p1**, **p1_conf**, and **p1_dog** under `image_predictions` need to be merged with `twitter_archive`
- **timestamp** contains both *DateTime* and *time*

In [403]:
#before cleaning, create a clean copy of each DataFrame
arc_clean = twitter_archive.copy()
img_clean = image_predictions.copy()
tweets_clean = tweet_fields.copy()

## CLEAN

### Quality

> **Issue 1:** `twitter_archive` contains 281 tweets that are missing from `image_predictions.`

**Define**

Drop the tweets_ids from `twitter_archive` that are not present in `image_predictions`.


**Code**

In [404]:
missing_clean = find_missing_tweet_ids(arc_clean, img_clean)

missing_ids = missing_clean.tweet_id
arc_clean = arc_clean[(arc_clean.tweet_id.isin(img_clean.tweet_id))]

**Test**

In [405]:
#verify that 281 entries have been dropped from arc_clean
twitter_archive.shape[0] - arc_clean.shape[0]

281

> **Issue 1a:** `twitter_archive` contains 22 tweets that are missing from `tweet_fields.` After dropping the previous entries, there should be 13 tweets to be dropped.

**Define**

Drop the tweets_ids from `twitter_archive` that are not present in `tweet_fields`.

From the previous step, there 9 dropped tweet IDs in common with the missing IDs in `tweet_fields.` Excluding these tweets that were already dropped, there should be 22 - 9 = **13 tweets** dropped after this step, accounting for a **total 294 entries dropped.** (281 + 13)


**Code**

In [406]:
missing_clean2 = find_missing_tweet_ids(arc_clean, tweets_clean)

missing_ids2 = missing_clean2.tweet_id
arc_clean = arc_clean[(arc_clean.tweet_id.isin(tweets_clean.tweet_id))]

**Test**

In [407]:
#verify that 281 + 13 = 294 entries have been dropped from arc_clean
twitter_archive.shape[0] - arc_clean.shape[0]

294

> **Issue 2:** retweeted data under `twitter_archive` indicates duplicated tweets.

**Define**

All nonduplicate tweets have null values under **retweeted_status_id**, **retweeted_status_user_id**, and **retweeted_status_timestamp.**

1. drop all rows from `twitter_archive` where **retweeted_status_user_id** is not null.
2. drop the 3 columns as they are no longer necessary.


**Code**

In [409]:
#1. drop entries where retweeted_status_user_id is not null
arc_clean = arc_clean[arc_clean.retweeted_status_user_id.isna()]

**Test**

In [410]:
#check that there are no duplicates
len(arc_clean[~(arc_clean.retweeted_status_user_id.isna())])

0

**Code**

In [411]:
#2. drop unnecessary columns
arc_clean = arc_clean.drop(columns = ['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'])

**Test**

In [412]:
#check that columns don't exist
arc_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1988 entries, 0 to 2355
Data columns (total 14 columns):
tweet_id                 1988 non-null int64
in_reply_to_status_id    23 non-null float64
in_reply_to_user_id      23 non-null float64
timestamp                1988 non-null object
source                   1988 non-null object
text                     1988 non-null object
expanded_urls            1988 non-null object
rating_numerator         1988 non-null int64
rating_denominator       1988 non-null int64
name                     1988 non-null object
doggo                    1988 non-null object
floofer                  1988 non-null object
pupper                   1988 non-null object
puppo                    1988 non-null object
dtypes: float64(2), int64(3), object(9)
memory usage: 233.0+ KB


In [586]:
#new size of DataFrame
arc_clean.shape[0]

1987

> **Issue 3:** unnecessary anchor tags under source.

**Define**

Use RegEx to reformat the strings under **source** column and only display the text between the anchor tags.


**Code**

In [413]:
arc_clean.source = arc_clean.source.str.extract('<a[^>]*>(.*?)</a>')

**Test**

In [414]:
arc_clean.source.value_counts()

Twitter for iPhone    1950
Twitter Web Client    28  
TweetDeck             10  
Name: source, dtype: int64

> **Issue 4:** erroneous datatypes under **in_reply_to_status_id**, **in_reply_to_user_id**, **retweeted_status_id**, **retweeted_status_user_id**, **timestamp**, and **retweeted_status_timestamp**.

**Define**

1. change data type of **in_reply_to_status_id** and **in_reply_to_user_id to integer**
2. change data type of **timestamp** to DateTime.

*The retweeted columns can be ignored now that they've been dropped.*


**Code**

In [415]:
#convert in_reply_to_status_id and in_reply_to_user_id to int64
arc_clean = arc_clean.fillna(0)

arc_clean.in_reply_to_status_id = arc_clean.in_reply_to_status_id.astype(np.int64)
arc_clean.in_reply_to_user_id = arc_clean.in_reply_to_user_id.astype(np.int64)

In [416]:
#convert timestamp to datetime
arc_clean.timestamp = pd.to_datetime(arc_clean.timestamp)

**Test**

In [417]:
arc_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1988 entries, 0 to 2355
Data columns (total 14 columns):
tweet_id                 1988 non-null int64
in_reply_to_status_id    1988 non-null int64
in_reply_to_user_id      1988 non-null int64
timestamp                1988 non-null datetime64[ns, UTC]
source                   1988 non-null object
text                     1988 non-null object
expanded_urls            1988 non-null object
rating_numerator         1988 non-null int64
rating_denominator       1988 non-null int64
name                     1988 non-null object
doggo                    1988 non-null object
floofer                  1988 non-null object
pupper                   1988 non-null object
puppo                    1988 non-null object
dtypes: datetime64[ns, UTC](1), int64(5), object(8)
memory usage: 233.0+ KB


In [418]:
arc_clean[~(arc_clean['in_reply_to_status_id'] == 0)]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
149,863079547188785154,667152164079423488,4196983835,2017-05-12 17:12:53+00:00,Twitter for iPhone,"Ladies and gentlemen... I found Pipsy. He may have changed his name to Pablo, but he never changed his love for the sea. Pupgraded to 14/10 https://t.co/lVU5GyNFen",https://twitter.com/dog_rates/status/863079547188785154/photo/1,14,10,None,None,None,None,None
184,856526610513747968,855818117272018944,4196983835,2017-04-24 15:13:52+00:00,Twitter for iPhone,"THIS IS CHARLIE, MARK. HE DID JUST WANT TO SAY HI AFTER ALL. PUPGRADED TO A 14/10. WOULD BE AN HONOR TO FLY WITH https://t.co/p1hBHCmWnA",https://twitter.com/dog_rates/status/856526610513747968/photo/1,14,10,None,None,None,None,None
251,844979544864018432,759099523532779520,4196983835,2017-03-23 18:29:57+00:00,Twitter for iPhone,"PUPDATE: I'm proud to announce that Toby is 236 days sober. Pupgraded to a 13/10. We're all very proud of you, Toby https://t.co/a5OaJeRl9B","https://twitter.com/dog_rates/status/844979544864018432/photo/1,https://twitter.com/dog_rates/status/844979544864018432/photo/1,https://twitter.com/dog_rates/status/844979544864018432/photo/1",13,10,None,None,None,None,None
565,802265048156610565,733109485275860992,4196983835,2016-11-25 21:37:47+00:00,Twitter for iPhone,"Like doggo, like pupper version 2. Both 11/10 https://t.co/9IxWAXFqze",https://twitter.com/dog_rates/status/802265048156610565/photo/1,11,10,None,doggo,None,pupper,None
1016,746906459439529985,746885919387574272,4196983835,2016-06-26 03:22:31+00:00,Twitter for iPhone,"PUPDATE: can't see any. Even if I could, I couldn't reach them to pet. 0/10 much disappointment https://t.co/c7WXaB2nqX",https://twitter.com/dog_rates/status/746906459439529985/photo/1,0,10,None,None,None,None,None
1018,746818907684614144,691416866452082688,4196983835,2016-06-25 21:34:37+00:00,Twitter for iPhone,Guys... Dog Jesus 2.0\n13/10 buoyant af https://t.co/CuNA7OwfKQ,https://twitter.com/dog_rates/status/746818907684614144/photo/1,13,10,None,None,None,None,None
1127,729838605770891264,729113531270991872,4196983835,2016-05-10 01:00:58+00:00,Twitter for iPhone,"""Challenge completed"" \n(pupgraded to 12/10) https://t.co/85dTK7XCXB",https://twitter.com/dog_rates/status/729838605770891264/video/1,12,10,None,None,None,None,None
1330,705786532653883392,703255935813287936,4196983835,2016-03-04 16:06:36+00:00,Twitter for iPhone,"Seriously, add us 🐶 11/10 for sad wet pupper https://t.co/xwPE9faVZR",https://twitter.com/dog_rates/status/705786532653883392/photo/1,11,10,None,None,None,pupper,None
1339,704871453724954624,667152164079423488,4196983835,2016-03-02 03:30:25+00:00,Twitter for iPhone,I found a forest Pipsy. 12/10 https://t.co/mIQ1KoVsmU,https://twitter.com/dog_rates/status/704871453724954624/photo/1,12,10,None,None,None,None,None
1356,703425003149250560,703041949650034688,4196983835,2016-02-27 03:42:44+00:00,Twitter for iPhone,"Really guys? Again? I know this is a rare Albanian Bingo Seal, but we only rate dogs. Only send in dogs... 9/10 https://t.co/6JYLpUmBrC",https://twitter.com/dog_rates/status/703425003149250560/photo/1,9,10,None,None,None,None,None


> **Issue 4:** inconsistent & inaccurate readings mapped from text to numerator & denominator fields (e.g. 24/7 being mistaken as a rating).

**Define**

Compare the untruncated **text** with each rating where tha denominator is not equal to 10, and correct them accordingly.


**Code**

In [419]:
arc_clean[arc_clean['rating_denominator'] != 10][['tweet_id', 'text', 'rating_numerator', 'rating_denominator']]

,tweet_id,text,rating_numerator,rating_denominator
433,820690176645140481,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,84,70
516,810984652412424192,Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx,24,7
902,758467244762497024,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,165,150
1068,740373189193256964,"After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ",9,11
1120,731156023742988288,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv,204,170
1165,722974582966214656,Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a,4,20
1202,716439118184652801,This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq,50,50
1228,713900603437621249,Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1,99,90
1254,710658690886586372,Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12,80,80
1274,709198395643068416,"From left to right:\nCletus, Jerome, Alejandro, Burp, &amp; Titson\nNone know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK",45,50


***Tracking***

Correct Tweet IDs (rating was recorded correctly):
- 820690176645140481
- 758467244762497024
- 731156023742988288
- 710658690886586372
- 709198395643068416
- 704054845121142784
- 697463031882764288	
- 684225744407494656
- 684222868335505415	
- 677716515794329600	
- 675853064436391936

Wrong Tweet IDs (along with correct ratings):
- 810984652412424192 (NA)
- 740373189193256964 (14/10)
- 722974582966214656 (13/10)
- 716439118184652801 (11/10)
- 713900603437621249 (99/90)
- 682962037429899265 (10/10)
- 666287406224695296 (9/10)

***So far, there are 7 corrections we have to make.***

In [420]:
ids_to_correct = [810984652412424192, 740373189193256964, 722974582966214656, 716439118184652801,
                  713900603437621249, 682962037429899265, 666287406224695296]

In [421]:
def correct_rating(tweet_id):
    if (tweet_id == ids_to_correct[0]):
        return [0, 0]
    elif (tweet_id == ids_to_correct[1]):
        return [14, 10]
    elif (tweet_id == ids_to_correct[2]):
        return [13, 10]
    elif (tweet_id == ids_to_correct[3]):
        return [11, 10]
    elif (tweet_id == ids_to_correct[4]):
        return [99, 90]
    elif (tweet_id == ids_to_correct[5]):
        return [10, 10]
    elif (tweet_id == ids_to_correct[6]):
        return [9, 10]
    else:
        return

In [422]:
for index, row in arc_clean[arc_clean.tweet_id.isin(ids_to_correct)].iterrows():
    rating = correct_rating(row['tweet_id'])
    if (rating is not None):
        arc_clean.at[index, 'rating_numerator'] = rating[0]
        arc_clean.at[index, 'rating_denominator'] = rating[1]

**Test**

In [423]:
arc_clean['rating_denominator'].value_counts()

10     1975
80     2   
50     2   
170    1   
150    1   
130    1   
120    1   
110    1   
90     1   
70     1   
40     1   
0      1   
Name: rating_denominator, dtype: int64

> **Issue 5:** some numerator values contain a decimal point (e.g. 9.75/10).

**Define**

Correct the numerators of the isolated tweet IDs from the text, then round them to account for the columns' int datatype.

I isolated these Tweet IDs through visual assessment.

**Code**

In [424]:
#make corrections to wrongly recorded numerators.
ids_wrong_numerator = [786709082849828864, 680494726643068929]

In [425]:
arc_clean.query('tweet_id == @ids_wrong_numerator[0] or tweet_id == @ids_wrong_numerator[1]')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
695,786709082849828864,0,0,2016-10-13 23:23:56+00:00,Twitter for iPhone,"This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",https://twitter.com/dog_rates/status/786709082849828864/photo/1,75,10,Logan,None,None,None,None
1712,680494726643068929,0,0,2015-12-25 21:06:00+00:00,Twitter for iPhone,Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,https://twitter.com/dog_rates/status/680494726643068929/photo/1,26,10,None,None,None,None,None


***Tracking:***

- 786709082849828864 (9.75/10)
- 680494726643068929 (11.26/10)

In [426]:
arc_clean.at[695, 'rating_numerator'] = round(9.75)
arc_clean.at[1712, 'rating_numerator'] = round(11.26)

**Test**

In [427]:
arc_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1988 entries, 0 to 2355
Data columns (total 14 columns):
tweet_id                 1988 non-null int64
in_reply_to_status_id    1988 non-null int64
in_reply_to_user_id      1988 non-null int64
timestamp                1988 non-null datetime64[ns, UTC]
source                   1988 non-null object
text                     1988 non-null object
expanded_urls            1988 non-null object
rating_numerator         1988 non-null int64
rating_denominator       1988 non-null int64
name                     1988 non-null object
doggo                    1988 non-null object
floofer                  1988 non-null object
pupper                   1988 non-null object
puppo                    1988 non-null object
dtypes: datetime64[ns, UTC](1), int64(5), object(8)
memory usage: 313.0+ KB


In [428]:
arc_clean.query('tweet_id == @ids_wrong_numerator[0] or tweet_id == @ids_wrong_numerator[1]')[['tweet_id','rating_numerator', 'rating_denominator']]

,tweet_id,rating_numerator,rating_denominator
695,786709082849828864,10,10
1712,680494726643068929,11,10


> **Issue 6:** rating_denominator has values besides 10

**Define**

Normalize the ratings using a common ratio so that they all have a common denominator of 10. Create 2 new columns for these normalized numerators and denominators.

e.g. for a rating of 200/50, calculate 200/50 = x/10 where x is the new normalized numerator.

$$\frac{numerator}{denominator} = \frac{x}{10} $$

$$ 10 * \frac{numerator}{denominator} = x $$

**Code**

In [429]:
for i, row in arc_clean.iterrows():
    if(row['rating_numerator'] != 0 and row['rating_denominator'] != 0):
        normalized_numerator = round(10 * (row['rating_numerator']/row['rating_denominator']))
        arc_clean.at[i, 'normalized_numerator'] = int(normalized_numerator)
    else:
        arc_clean.at[i, 'normalized_numerator'] = 0

In [430]:
#convert normalized_numerator to int
arc_clean.normalized_numerator = arc_clean.normalized_numerator.astype(np.int64)

**Test**

In [431]:
#check ratings where denominators are not 10
arc_clean.query('rating_denominator != 10')[['tweet_id', 'rating_numerator', 'rating_denominator', 'normalized_numerator']]

,tweet_id,rating_numerator,rating_denominator,normalized_numerator
433,820690176645140481,84,70,12
516,810984652412424192,0,0,0
902,758467244762497024,165,150,11
1120,731156023742988288,204,170,12
1228,713900603437621249,99,90,11
1254,710658690886586372,80,80,10
1274,709198395643068416,45,50,9
1351,704054845121142784,60,50,12
1433,697463031882764288,44,40,11
1634,684225744407494656,143,130,11


In [432]:
#for denominators with 10, verify that numerators are unchanged
arc_clean.query('rating_denominator == 10')[['tweet_id', 'rating_numerator', 'rating_denominator', 'normalized_numerator']].sample(10)

,tweet_id,rating_numerator,rating_denominator,normalized_numerator
1027,746056683365994496,11,10,11
1560,688519176466644993,12,10,12
1017,746872823977771008,11,10,11
970,750117059602808832,10,10,10
540,806542213899489280,11,10,11
2290,667171260800061440,10,10,10
1656,683357973142474752,10,10,10
1755,678774928607469569,11,10,11
33,885984800019947520,12,10,12
297,837110210464448512,13,10,13


> **Issue 7:** invalid dog names such as a, an, quite, by, actually, such, not, one.

**Define**

Remove all lower case strings under the **name** field. Invalid names all share 1 common property: the first character is always lower case.

**Code**

In [436]:
name_mask = arc_clean.name.str.islower()
arc_clean['name'] = arc_clean['name'].where(~name_mask, other='None')

**Test**

In [446]:
#verify that all names are capitalized
arc_clean.name.str.islower().value_counts()

False    1988
Name: name, dtype: int64

> **Issue 8:** inconsistent timestamp format between `twitter_archive` and `tweet_fields`.

**Define**

Drop the **created_at** column under `tweet_fields`. The format for timestamp under `twitter_archive` is recognized as legitimate by Panda's datetime().

**Code**

In [458]:
tweets_clean = tweets_clean.drop(columns = 'created_at')

**Test**

In [459]:
tweets_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2334 entries, 0 to 2333
Data columns (total 3 columns):
tweet_id          2334 non-null int64
favorite_count    2334 non-null int64
retweet_count     2334 non-null int64
dtypes: int64(3)
memory usage: 54.8 KB


> **Issue 10:** there is one entry with no rating.

**Define**

Drop the entry in `arc_clean` where I previously set the numerator and denominator to 0 as a placeholder for NaN.

**Code**

In [467]:
drop_index = arc_clean.query('rating_denominator == 0').index[0]
arc_clean = arc_clean.drop(drop_index)

**Test**

In [470]:
arc_clean.rating_denominator.value_counts()

10     1975
80     2   
50     2   
170    1   
150    1   
130    1   
120    1   
110    1   
90     1   
70     1   
40     1   
Name: rating_denominator, dtype: int64

### Tidiness

> **Issue 1:** **doggo**, **floofer**, **pupper**, and **puppo** all are not under 1 category variable.

**Define**

1. Create a new column **dog_stage** and set values to where prior dog stage columns are not equal to None. Separate dog stages by comma if a tweet falls under more than 1 stage.
2. Drop **doggo**, **floofer**, **pupper**, and **puppo** columns.


**Code**

In [547]:
#1. create dog_stage column with dog stages
arc_clean['dog_stage'] = 'None'

In [571]:
for i, row in arc_clean.iterrows():
    stage_string = 'None'
    if row['doggo'] != 'None':
        stage_string += 'doggo, '
    if row['floofer'] != 'None':
        stage_string += 'floofer, '
    if row['pupper'] != 'None':
        stage_string += 'pupper, '
    if row['puppo'] != 'None':
        stage_string += 'puppo'
        
    arc_clean.at[i, 'dog_stage'] = stage_string

In [572]:
all_mask = mask_all_stages(arc_clean)

#strip commas and replace None values for entries with >= 1 dog stage
arc_clean.dog_stage[all_mask] = arc_clean[all_mask].dog_stage.str.replace('None', '').str.strip(', ')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [578]:
#drop doggo, floofer, pupper, and puppo stages
arc_clean.drop(columns = ['doggo', 'floofer', 'pupper', 'puppo'], inplace = True)

**Test**

In [573]:
single_mask = mask_single_stages(arc_clean)
duplicate_mask = mask_duplicate_stages(arc_clean)

In [574]:
arc_clean[all_mask].sample(10).iloc[:, -1]

1192    pupper       
977     doggo        
191     doggo, puppo 
1590    pupper       
802     pupper       
1063    doggo, pupper
483     pupper       
1059    pupper       
1250    pupper       
240     doggo        
Name: dog_stage, dtype: object

In [575]:
arc_clean[duplicate_mask].iloc[:, -1]

191     doggo, puppo  
200     doggo, floofer
460     doggo, pupper 
531     doggo, pupper 
565     doggo, pupper 
575     doggo, pupper 
705     doggo, pupper 
889     doggo, pupper 
956     doggo, pupper 
1063    doggo, pupper 
1113    doggo, pupper 
Name: dog_stage, dtype: object

In [576]:
arc_clean[single_mask].sample(10).iloc[:, -1]

56      pupper 
1157    pupper 
1279    pupper 
582     floofer
1617    pupper 
1122    pupper 
429     doggo  
1000    pupper 
1049    pupper 
2015    pupper 
Name: dog_stage, dtype: object

In [577]:
#check that None is preserved for entries with no recorded dog stages
arc_clean.sample(10).iloc[:,-1]

1267    None   
1384    None   
1747    None   
1613    None   
1091    floofer
1152    pupper 
2188    None   
1490    None   
389     doggo  
1131    None   
Name: dog_stage, dtype: object

In [579]:
#check that columns have dropped
arc_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1987 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id                 1987 non-null int64
in_reply_to_status_id    1987 non-null int64
in_reply_to_user_id      1987 non-null int64
timestamp                1987 non-null datetime64[ns, UTC]
source                   1987 non-null object
text                     1987 non-null object
expanded_urls            1987 non-null object
rating_numerator         1987 non-null int64
rating_denominator       1987 non-null int64
name                     1987 non-null object
normalized_numerator     1987 non-null int64
dog_stage                1987 non-null object
dtypes: datetime64[ns, UTC](1), int64(6), object(5)
memory usage: 281.8+ KB


> **Issue 2:** favorite_count and retweet_count fields under `tweets_clean` need to be merged with `arc_clean.`

**Define**

1. Perform an inner merge between `tweets_clean` and `arc_clean` on **tweet_id**, and add **favorite_count** & **retweet_count** fields from `tweets_clean`.

**Code**

In [584]:
arc_clean = pd.merge(arc_clean, tweets_clean, on = 'tweet_id')

**Test**

In [587]:
#make sure size is preserved
arc_clean.shape[0]

1987

> **Issue 3:** **p1**, **p1_conf**, and **p1_dog** under `image_predictions` need to be merged with `twitter_archive`

**Define**

1. Perform an inner merge between `tweets_clean` and `arc_clean` on **tweet_id**, and add **favorite_count** & **retweet_count** fields from `tweets_clean`.

**Code**


## Analysis

- On average, do certain dog breeds tend to get higher ratings?

- Which time of day does the follower count for the WeRateDogs page tend to spike?

# Sources

https://stackoverflow.com/questions/28384588/twitter-api-get-tweets-with-specific-id

https://stackoverflow.com/questions/27900451/convert-tweepy-status-object-into-json

https://stackoverflow.com/questions/7370801/measure-time-elapsed-in-python

- https://github.com/gouravaich/wrangle-analyze-weratedogs-twitter/blob/master/Solution.ipynb (this is where I got the idea to add an error file)

https://stackoverflow.com/questions/3395138/using-multiple-arguments-for-string-formatting-in-python-e-g-s-s

https://stackoverflow.com/questions/25351968/how-to-display-full-non-truncated-dataframe-information-in-html-when-convertin